In [1]:
#!pip install -U selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import xlwings as xw
import datetime 
import time
from tkinter import filedialog as fd
import tkinter as tk
#from PIL import ImageTk, Image
import re
import threading
import os
import win32file
import sys
import pythoncom

In [2]:
def Days(d1,d2): # format like this: 111/01/01
    try:
        a = d1.split("/")
        d1 = str(int(a[0]) + 1911) + "/" + str(int(a[1])) + "/" + str(int(a[2]))
        b = d2.split("/")
        d2 = str(int(b[0]) + 1911) + "/" + str(int(b[1])) + "/" + str(int(b[2]))    

        # （1）先將字串-->時間格式date
        date1 = datetime.datetime.strptime(d1, "%Y/%m/%d").date()  ##datetime.date(2018, 1, 6)
        date2 = datetime.datetime.strptime(d2, "%Y/%m/%d").date()  ##datetime.date(2018, 1, 9)
        # （2）計算兩個日期date的天數差
        Days = (date2 - date1).days 
        return Days
    except:
        Days = "GG"  #out of date range
        return Days
        

In [3]:
# 判斷檔案是否開啟中
def is_used(file_name):
    try:
        vHandle = win32file.CreateFile(file_name, win32file.GENERIC_READ, 0, None, win32file.OPEN_EXISTING, win32file.FILE_ATTRIBUTE_NORMAL, None)
        return int(vHandle) == win32file.INVALID_HANDLE_VALUE
    except:
        return True
    finally:
        try:
            win32file.CloseHandle(vHandle)
        except:
            pass

In [4]:
def selectPath():   
    global dir_path
    dir_path = fd.askdirectory()
    if dir_path:     
        result_label.configure(text="儲存路徑:\n{}".format(dir_path))

    else:
        result_label.configure(text="未選擇儲存路徑")
        del dir_path

In [5]:
def search_settings(t_name,t_start,t_end): #標案名稱, 起始日期, 截止日期
    driver.get(URL) 
    #機關名稱
    org_name = driver.find_element(By.NAME,"orgName")
    org_name.clear()
    org_name.send_keys("") #ex: 總統府
    # #機關代碼
    org_id = driver.find_element(By.NAME,"orgId")
    org_id.clear()
    org_id.send_keys("") #ex: 2
    #標案名稱
    tender_name = driver.find_element(By.NAME,"tenderName")
    tender_name.clear()
    tender_name.send_keys(t_name) #ex: 市場研究
    #標案案號
    tender_id = driver.find_element(By.NAME,"tenderId")
    tender_id.clear()
    tender_id.send_keys("") 

    #招標類型 
    #0:招標公告 1:公開徵求 2:公開閱覽 3:政府採購預告
    tender_type = Select(driver.find_element(By.NAME,"tenderType"))
    tender_type.select_by_index(0)
    #tender_type.select_by_visible_text(u"招標公告")

    #招標方式
    #-----------------------------
    #0:招標公告: 
    #各式招標公告,公開招標, 公開取得電子報價單, 公開取得報價單或企劃書, 經公開評選或公開徵求之限制性招標,
    #選擇性招標(建立合格廠商名單), 選擇性招標(建立合格廠商名單後續邀標), 選擇性招標(個案), 電子競價, 限制性招標(未經公開評選或公開徵求)
    #-----------------------------
    #3:政府採購預告:
    #各式預定招標方式,公開招標,公開取得報價單或企劃書公告,限制性招標(公開評選、公開徵求),選擇性招標(建立合格廠商名單)
    #選擇性招標(建立合格廠商後續邀標),選擇性招標(個案)
    #-----------------------------
    tender_way = Select(driver.find_element(By.NAME,"tenderWay"))
    #tender_way.select_by_index(0)
    tender_way.select_by_visible_text(u"各式招標公告")

    #公告日期
    tender_today = driver.find_element(By.ID,"basicIsNowDateTypeId") #當日
    # tender_today.click()
    tender_range = driver.find_element(By.ID,"basicIsSpdtDateTypeId") #等標期內
    # tender_range.click()
    tender_date = driver.find_element(By.ID,"basicIsDateDateTypeId") #日期範圍
    tender_date.click()
    start_date = driver.find_element(By.XPATH,'//*[@id="tenderStartDateArea"]/div/div/input[1]') #起始日期
    start_date.clear()
    start_date.send_keys(t_start) #ex:"111/03/01"
    start_confirm = driver.find_element(By.XPATH,'//*[@id="calendarDiv"]/div[1]/div[2]/div/span[3]') #起始日期確認按鈕
    start_confirm.click()
    
    
    end_date = driver.find_element(By.XPATH,'//*[@id="tenderEndDateArea"]/div/div/input[1]') #結束日期
    end_date.clear()
    end_date.send_keys(t_end) #ex:"111/03/15" or today()
    end_confirm = driver.find_element(By.XPATH,'//*[@id="calendarDiv"]/div[1]/div[2]/div/span[3]') #起始日期確認按鈕
    end_confirm.click()
    

    #標的分類
    catagory = "勞務"
    engineering = driver.find_element(By.ID,"basicRadProctrgCate1") #工程
    finance = driver.find_element(By.ID,"basicRadProctrgCate2") #財務
    labour = driver.find_element(By.ID,"basicRadProctrgCate3") #勞務
    unlimit = driver.find_element(By.ID,"basicRadProctrgCate4") #不限    

    labour.click()
    

    #適用之條約或協定
    GPA = driver.find_element(By.ID,"gpa")
    ANZTEC = driver.find_element(By.ID,"anztec")
    ASTEP = driver.find_element(By.ID,"astep")
    treaties = [] #勾選 ["GPA","ANZTEC","ASTEP"]
    if ("GPA" in treaties and not(GPA.is_selected())) or ((not("GPA" in treaties)) and GPA.is_selected()):
        GPA.click()
    if ("ANZTEC" in treaties and not(ANZTEC.is_selected())) or ((not("ANZTEC" in treaties)) and ANZTEC.is_selected()):
        ANZTEC.click()
    if ("ASTEP" in treaties and not(ASTEP.is_selected())) or ((not("ASTEP" in treaties)) and ASTEP.is_selected()):
        ASTEP.click()                


In [6]:
def get_data():

    source = driver.page_source 

    
    soup = BeautifulSoup(source, 'html.parser')
    contents = soup.find_all('tr', class_ = ['tb_b2','tb_b3'])

    
    for content in contents:                
        cont = content.find_all('td')
        
        #公告日期
        t_start_date = "'" + cont[6].text.strip() if cont[6] else ''
        #print(t_start_date)        
        #截止日期
        t_end_date = "'" + cont[7].text.strip() if cont[7] else ''
        #print(t_end_date)      
        #機關名稱
        o_name = "'" + cont[1].text.strip() if cont[1] else ''
        #print(o_name) 

        #標案案號
        t_id = "'" + cont[2].text.strip().replace('\n','').replace('\t','').split(' ')[0] if cont[2] else ''
        #print(t_id)
        
        
        #標案名稱
        t_name = "'" + cont[2].text.strip().replace('\n','').replace('\t','').split(' ')[-1] if cont[2] else ''
        #print(t_name)



        #傳輸次數
        t_times = "'" + cont[3].text.strip().replace('\n','') if cont[3] else ''
        #標案案號:傳輸次數
        t_id_times = t_id + ":" + t_times.strip("'")
        
        #print(t_times)
        #print(t_id_times)          
        
        #預算
        t_budget = "'" + cont[8].text.strip().replace(' ','') if cont[8] else ''
        #print(t_budget)


        #連結
        t_link = "https://web.pcc.gov.tw/" + content.find_all('a')[0].get('href') if content.find_all('a')[0] else ''
        #print(t_link)
        
        # BOOK中sheet的数量
        count = wb.sheets.count
        # 取到所有sheet的name
        name_list = [wb.sheets[i].name for i in range(0, count)]        
        #創建月份sheet        
        public_month = str(int(t_start_date.split('/')[1])) + "月"
        
        if public_month not in name_list:
            wb.sheets.add(public_month)
            sheet = wb.sheets[public_month]
            sheet.range('A1').value = [['標案案號','投標單位 = 機關名稱','標案名稱','預算','截止投標','連結網址','標案案號:傳輸次數']]
            sheet.range('A1:G1').color = '#99C0EC'
            
        else:
            pass

        #進入當月sheet填入資料
        if public_month in ["1月","2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月"]:            
            sheet = wb.sheets[public_month]
            current_rows = sheet.range('A' + str(sheet.cells.last_cell.row)).end('up').row #last row of the sheet
            #ID及傳輸次數不重複才放入excel:
            if t_id_times.lstrip("'") not in sheet.range("G2:G{}".format(current_rows)).value:
                # scenario_1 機關名稱為:交通部公路總局,衛生福利部,財團法人中華民國對外貿易發展協會,
                #                      勞動部勞動力發展署,交通部觀光局,經濟部國營事業委員會,臺灣菸酒股份有限公司
                orgs = ["交通部公路總局","衛生福利部","財團法人中華民國對外貿易發展協會","勞動部勞動力發展署","交通部觀光局","經濟部國營事業委員會","臺灣菸酒股份有限公司"]
                scenario_1 = False
                for o in orgs:
                    if o in o_name.strip("'"):
                        scenario_1 = True
                    
                # scenario_2 非scenario_1 且 新台幣預算達50萬以上
                t_budget = int(t_budget.lstrip("'").replace(",",'')) if (t_budget.lstrip("'").replace(",",'')) != '' else 0
                scenario_2 = not(scenario_1) and (t_budget >= 500000)
                if scenario_1 or scenario_2:                    
                    sheet.range("A{}".format(current_rows+1)).value = [[t_id,o_name,t_name,t_budget,t_end_date]]
                    sheet.range("F{}".format(current_rows+1)).add_hyperlink(t_link)
                    sheet.range("G{}".format(current_rows+1)).value = t_id_times
                    sheet.range("A{}".format(current_rows+1)).api.HorizontalAlignment = -4131 # ID置左對齊 
                    if scenario_1:                        
                        sheet.range("A{}:G{}".format(current_rows+1,current_rows+1)).color = "#A6FFA6"
            else:
                print("重複ID及傳輸次數: ",t_id_times.lstrip("'"))
    
    #調整sheet欄寬
    for sht in wb.sheets:
        sht.autofit(axis='c')   

In [7]:
def start_task(): 
    
    try:    
        if t_start and t_end and (re.match(pattern, t_start)) and (re.match(pattern, t_end)) and len(t_start) == 9 and len(t_end) == 9:

            result_label.configure(text="資料載入中...")

            for name in t_name:
                search_settings(name,t_start,t_end)
                #search
                search = driver.find_element(By.XPATH,'//*[@id="basicTenderSearchForm"]/div/div[15]/div/div')
                search.click()
                #wait for 20 seconds until find class_name="__" (招標查詢)

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/div[3]')))
                time.sleep(5)

                ## 抓取頁面資料
                done = 0
                while done==0:

                    get_data()
                    time.sleep(3)
                    try:
                        next_page = driver.find_element(By.XPATH,'//*[@id="pagelinks"]/a[1]')
                        next_page.click()

                    except:
                        print("This is the last page of search results for the: ", name)
                    done = 1
        return 1


    except:
        result_label.configure(text="Something Wrong\n請開啟工作管理員並將 Excel 結束工作")


In [ ]:
import threading



class Threader(threading.Thread):

    def __init__(self, *args, **kwargs):

            threading.Thread.__init__(self, *args, **kwargs)
            self.daemon = True
            self.start() 
            

    def run(self):       

        pythoncom.CoInitialize()
        ck_path = 0
        ck_date = 0        
        ck_browser = 0
        ck_env = 0

        
        #確認儲存路徑及讀取檔案--------------------------------------------------------------------------        
        if "dir_path" in globals():
            ck_path = 1
                
        else:
            result_label.configure(text="請選擇儲存路徑")              
        #--------------------------------------------------------------------------------------------------
        if ck_path == 1:                                
            
            #年月日格式確認區--------------------------------------------------------------------------------------
            global pattern, t_name, t_start, t_end
            pattern = "(\d{3}/([0][1-9]|[1][0-2])/([0][1-9]|[1][0-9]|[2][0-9]|[3][0-2]))"            
            t_name = ["調查","訪問","問卷","統計"]
            t_start =  str(s_date_entry.get())
            t_end = str(e_date_entry.get())
            
                        
            if t_start and t_end and (re.match(pattern, t_start)) and (re.match(pattern, t_end)) and len(t_start) == 9 and len(t_end) == 9:
                if (type(Days(t_start,t_end)) == int) and (Days(t_start,t_end) >= 0) and (Days(t_start,t_end) <= 93):                    
                    ck_date = 1 #正確格式                    
                elif (type(Days(t_start,t_end)) == int) and (Days(t_start,t_end) > 93): 
                    result_label.configure(text="日期搜尋區間請勿超過三個月")    
                elif (type(Days(t_start,t_end)) == int) and (Days(t_start,t_end) < 0):
                    result_label.configure(text="日期搜尋區間請由早至晚")
                elif str(Days(t_start,t_end)) == "GG":
                    result_label.configure(text="搜尋日不存在")
                    
            else:
                if (not t_start) or (not(t_end)):

                    result_label.configure(text="請輸入搜尋日期區間")
                else:

                    result_label.configure(text="請輸入正確日期格式")

            #---------------------------------------------------------------------------------------------------            
            
            if ck_date == 1:
                #瀏覽器driver 確認區-------------------------------------------------------------------------------------------------

                result_label.configure(text="執行中...")

                global driver

                object_existed = False
                if "driver" in globals():
                    if driver is not None: #瀏覽器driver存在時執行確認
                        try:
                            driver.execute_script('javascript:void(0);')
                            object_existed = True
                            if radioValue_a.get() == 1:
                                driver = webdriver.Chrome(service=s) #set argument: "options=chrome_options" to hide the window
                            elif radioValue_a.get() == 2:
                                driver = webdriver.Chrome(service=s,options=chrome_options) #set argument: "options=chrome_options" to hide the window



                            result_label.configure(text="瀏覽器已開啟")
                            print("瀏覽器已開啟")
                            ck_browser = 1
                        except:
                            # webdriver 要求瀏覽器執行Javescript時出現異常
                            try:
                                result_label.configure(text="正為您重啟瀏覽器中...")
                                print("正為您重啟瀏覽器中...")
                                object_existed = False                    
                                driver.quit()

                            finally:
                                driver = None

                        finally:
                            pass

                if not object_existed:
                    result_label.configure(text="瀏覽器開啟中")            
                    print("瀏覽器開啟中")

                    chrome_options = Options()
                    chrome_options.add_argument("--headless") #隱藏視窗背景執行
                    chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36') #防止識別

                    s = Service(PATH)                     
                                       
                    if radioValue_a.get() == 1:                                                
                        driver = webdriver.Chrome(service=s)                    
                    elif radioValue_a.get() == 2:
                        driver = webdriver.Chrome(service=s,options=chrome_options) #set argument: "options=chrome_options" to hide the window
                    #政府電子採購網_首頁>招標查詢
                    driver.get(URL) 
                    print(driver.title)
                    ck_browser = 1
                #---------------------------------------------------------------------------------------------------------------------             
                if ck_date == 1 and ck_path == 1 and ck_browser == 1:

                    #創建excel-------------------------------------------------------------------------------------------
                    # 檢查檔案是否存在
                    global file_path             
                    global wb

                    file_path = dir_path + "/" + "GOV_Tender.xlsx" 
                    if os.path.isfile(file_path): #已存在則開啟
                        if is_used(file_path): #開啟中則pass
                            ck_env = 1                            
                        else: #未開啟則開啟
                            result_label.configure(text="檔案已存在，正為您開啟: GOV_Tender.xlsx。")
                            if radioValue_b.get() == 1:
                                app = xw.App(visible=True, add_book=False) #顯示excel
                            elif radioValue_b.get() == 2:
                                app = xw.App(visible=False, add_book=False) #隱藏excel
                            wb = app.books.open(file_path) # 打開Excel文件    
                            result_label.configure(text="檔案已開啟: GOV_Tender.xlsx。")
                            ck_env = 1

                    else: #不存在則創建
                        result_label.configure(text="正為您創建檔案: GOV_Tender.xlsx") 
                        if radioValue_b.get() == 1:
                            app = xw.App(visible=True, add_book=False) #顯示excel
                        elif radioValue_b.get() == 2:
                            app = xw.App(visible=False, add_book=False) #隱藏excel
                        try:    
                            wb = app.books.add()
                            wb.save(file_path)  
                            result_label.configure(text="檔案已開啟: GOV_Tender.xlsx")
                            ck_env = 1 
                        except:
                            result_label.configure(text="Something Wrong\n請開啟工作管理員並將 Excel 結束工作")
                            
                    #----------------------------------------------------------------------------------------------------        
        
        
                    if ck_env == 1 :
                       #資料抓取----------------------------------------------------------------------------------
                        result_label.configure(text="取得資料中...")
                        time.sleep(3)

                        if start_task() == 1:
                            try:
                                #儲存並關閉excel
                                wb.save()
                                wb.close()
                                app.kill()
                                result_label.configure(text="執行完成\n{}".format(file_path))
                                print("執行完成")
                                pythoncom.CoUninitialize()
                            except:
                                result_label.configure(text="Something Wrong\n請開啟工作管理員並將 Excel 結束工作")
                                
                        else:
                            result_label.configure(text="Something Wrong\n請開啟工作管理員並將 Excel 結束工作")


                    else:
                        result_label.configure(text="Something Wrong\n請開啟工作管理員並將 Excel 結束工作")

        

#main thread
if __name__ == '__main__':

    """
    GUI 介面
    """
    if "dir_path" in globals():
        del dir_path


    URL = "https://web.pcc.gov.tw/pis/"
    PATH = "chromedriver.exe"



    window = tk.Tk()
    window.title('GOV Tender Web Crawler')
    window.geometry('410x620')
    window.configure(background='#DFFFDF')

    header_label = tk.Label(window, text='GOV Tender Web Crawler', width=24, height=2, bg='#000000', fg='#28FF28')
    header_label.config(font='helvetica 18 bold')
    header_label.place(x = 20, y = 20)


    #標案公告日期
    s_date_frame = tk.Frame(window) 
    s_date_label = tk.Label(s_date_frame, bg='#DFFFDF', text='標案搜尋日期區間', font='helvetica 10 bold', width=15, anchor='e')
    s_date_label.pack(side=tk.LEFT)
    s_date_entry = tk.Entry(s_date_frame, width=13)
    s_date_entry.pack(side=tk.LEFT)
    s_date_frame.place(x = 20, y = 100)

    e_date_frame = tk.Frame(window)
    e_date_label = tk.Label(e_date_frame, bg='#DFFFDF', text='－', font='helvetica 10 bold', width=0, anchor='e')
    e_date_label.pack(side=tk.LEFT)
    e_date_entry = tk.Entry(e_date_frame, width=13)
    e_date_entry.pack(side=tk.LEFT)
    e_date_frame.place(x = 240, y = 100)
    example_label = tk.Label(window, bg='#DFFFDF', text = "Example: 111/01/01 - 111/01/31", font='helvetica 9 bold',fg="#696969")
    example_label.place(x = 20, y = 123)

    # Function Button select path
    path_btn = tk.Button(window, activebackground='azure3', text='選擇儲存路徑', width=11,height=2, command=selectPath)
    path_btn.place(x = 25, y = 142)

    # Function Button start
    start_btn = tk.Button(window, activebackground='azure3', text='Start', width=11,height=3, command=lambda: Threader(name='Thread-name'))
    start_btn.place(x = 300, y = 140)

    #備註
    note_label = tk.Label(window, bg='#DFFFDF', text = "備註:檔名為 GOV_Tender.xlsx", font='helvetica 10 bold', fg="#EA0000")
    note_label.place(x = 20, y = 183)

    #searched by
    keyword_label = tk.Label(window, bg='#DFFFDF', text = "Searched by:", font='helvetica 11 bold')
    keyword_label.place(x = 19, y = 203)

    #標案名稱關鍵字
    keyword_frame = tk.Frame(window)
    keyword_label_1 = tk.Label(window, bg='#DFFFDF', text = "1.標案名稱關鍵字: ", font='helvetica 10 bold')
    keyword_label_1.place(x = 20, y = 230)
    keyword_label_2 = tk.Label(window, bg='#DFFFDF', text = "調查、訪問、問卷、統計", fg="#0000E3", font='helvetica 10 bold')
    keyword_label_2.place(x = 150, y = 230)

    #標的分類
    catg_label = tk.Label(window, bg='#DFFFDF', text = "2.標的分類為勞務類", font='helvetica 10 bold')
    catg_label.place(x = 20, y = 260)

    #機關單位關鍵字1
    org_label_1 = tk.Label(window, bg='#DFFFDF', text = "3.特定機關單位 (關鍵字部分符合):", font='helvetica 10 bold')
    org_label_1.place(x = 20, y = 290)

    #機關單位關鍵字2
    org_label_2 = tk.Label(window, bg='#DFFFDF', text = 
    '''    交通部公路總局\n
    衛生福利部\n
    財團法人中華民國對外貿易發展協會\n
    勞動部勞動力發展署\n
    交通部觀光局\n
    經濟部國營事業委員會\n
    臺灣菸酒股份有限公司''',font='helvetica 9 bold', fg="#0000E3", width=38,justify = 'left')
    org_label_2.place(x = 0, y = 315)

    #其他條件
    other_label = tk.Label(window, bg='#DFFFDF', text = "4.其他機關單位且預算50萬以上", font='helvetica 10 bold')
    other_label.place(x = 20, y = 520)




    #show browser or not    
    browser_label = tk.Label(window, bg='#DFFFDF', text = "瀏覽器顯示", font='helvetica 10 bold')
    browser_label.place(x = 300, y = 270)    

    radioValue_a = tk.IntVar() 
    rdioOne_a = tk.Radiobutton(window, text='Yes', bg='#DFFFDF',indicatoron = 0,width=6,height=3,font='helvetica 10 bold',
                                 variable=radioValue_a, value=1) 
    rdioTwo_a = tk.Radiobutton(window, text='No', bg='#DFFFDF',indicatoron = 0,width=6,height=3,font='helvetica 10 bold',
                                 variable=radioValue_a, value=2) 
    rdioOne_a.grid(column=0, row=0)
    rdioTwo_a.grid(column=2, row=0)    
    rdioOne_a.place(x=290, y = 300)
    rdioTwo_a.place(x=340, y = 300)
    rdioTwo_a.select() #預設隱藏瀏覽器

    #show excel or not    
    excel_label = tk.Label(window, bg='#DFFFDF', text = "Excel 顯示", font='helvetica 10 bold')
    excel_label.place(x = 300, y = 380)    

    radioValue_b = tk.IntVar() 
    rdioOne_b = tk.Radiobutton(window, text='Yes', bg='#DFFFDF',indicatoron = 0,width=6,height=3,font='helvetica 10 bold',
                                 variable=radioValue_b, value=1) 
    rdioTwo_b = tk.Radiobutton(window, text='No', bg='#DFFFDF',indicatoron = 0,width=6,height=3,font='helvetica 10 bold',
                                 variable=radioValue_b, value=2) 
    rdioOne_b.grid(column=0, row=0)
    rdioTwo_b.grid(column=2, row=0)    
    rdioOne_b.place(x=290, y = 410)
    rdioTwo_b.place(x=340, y = 410)
    rdioTwo_b.select() #預設隱藏瀏覽器




    result_label = tk.Label(window,  width=45, height=4, wraplength=300,font=('System', 14), bg='#DFFFDF', fg='blue')  
    result_label.place(x = 20, y = 543)

    window.mainloop()    

    try:
        driver.quit() #關閉瀏覽器driver
    except:
        pass




瀏覽器開啟中
政府電子採購網
重複ID及傳輸次數:  111S017:01
重複ID及傳輸次數:  FL111029:01
重複ID及傳輸次數:  111-008:01
重複ID及傳輸次數:  11101271:01
重複ID及傳輸次數:  NCCL110032:01
重複ID及傳輸次數:  11101275:01
重複ID及傳輸次數:  111B016-V6:01
重複ID及傳輸次數:  2571100003:01
重複ID及傳輸次數:  AF-111-04:01
重複ID及傳輸次數:  111Y087:01
重複ID及傳輸次數:  B11119:01
重複ID及傳輸次數:  111207:01
重複ID及傳輸次數:  1110004603:01
重複ID及傳輸次數:  KM1111008:01
重複ID及傳輸次數:  111A070:01
重複ID及傳輸次數:  C1120126:01
重複ID及傳輸次數:  1114E002:01
重複ID及傳輸次數:  1114E001:01
重複ID及傳輸次數:  111-C-46:01
重複ID及傳輸次數:  11102SA010:01
重複ID及傳輸次數:  ndc111005:01
重複ID及傳輸次數:  L0210P2081V:03
重複ID及傳輸次數:  1111B1C03:01
重複ID及傳輸次數:  B1110108-1A:03
重複ID及傳輸次數:  111A135:01
重複ID及傳輸次數:  1110010439:01
重複ID及傳輸次數:  111010006:01
重複ID及傳輸次數:  C1110124-1:01
重複ID及傳輸次數:  111A009-U:02
重複ID及傳輸次數:  111-C-42:01
重複ID及傳輸次數:  111405A303:01
重複ID及傳輸次數:  111D03:01
重複ID及傳輸次數:  1113005:01
重複ID及傳輸次數:  1110121021A:01
重複ID及傳輸次數:  111LL034:01
重複ID及傳輸次數:  111LL033:01
重複ID及傳輸次數:  111LL035:01
重複ID及傳輸次數:  111002:01
重複ID及傳輸次數:  1110705:01
重複ID及傳輸次數:  1110210B:01
重複ID及傳輸次數

# Test Area

In [ ]:
URL = "https://web.pcc.gov.tw/pis/"
PATH = "chromedriver.exe"
chrome_options = Options()
chrome_options.add_argument("--headless") #隱藏視窗背景執行
#driver = webdriver.Chrome(service=s, options=chrome_options) #set argument: "options=chrome_options" to hide the window
driver = webdriver.Chrome(service=s) #set argument: "options=chrome_options" to hide the window

In [ ]:
search_settings("調查","111/04/20","111/05/30")

In [ ]:
search = driver.find_element(By.XPATH,'//*[@id="basicTenderSearchForm"]/div/div[15]/div/div')
search.click()

In [ ]:
app = xw.App(visible=True, add_book=False)
wb = app.books.open(r"C:\Users\Mason\Downloads\GOV_Tender.xlsx")

In [ ]:
done = 0
while done==0:
    get_data()
    time.sleep(3)
    try:
        next_page = driver.find_element(By.XPATH,'//*[@id="pagelinks"]/a[1]')
        next_page.click()

    except:
        print("This is the last page of search results for the: ", 'xxx')
        done = 1

In [ ]:
# next_page = driver.find_element(By.XPATH,'//*[@id="pagelinks"]/a[1]')
# next_page.click()